In [1]:
# Read the data file
import torch
import torchvision
import numpy as np

### Read flattened, processed data
import pandas as pd
annotation = pd.read_csv("data/annotations.csv")

## Train-test split
from sklearn.model_selection import train_test_split
# train_fash, test_fash = train_test_split(fash_mnist, test_size=0.1, random_state=5)

### Separate the label column (outcome)
# train_y = train_fash['y']
# train_X = train_fash.drop(['y'], axis=1)
# test_y = test_fash['y']
# test_X = test_fash.drop(['y'], axis=1)

### Convert to numpy array then reshape to 900 by 28 by 28
# mnist_unflattened = train_X.to_numpy()
# mnist_unflattened = mnist_unflattened.reshape(900,28,28)

## Convert to tensor
# mnist_tensor = torch.from_numpy(mnist_unflattened)

## Check shape of the first image
# print(mnist_tensor[0,:,:].shape)

In [ ]:
# annotation.head()
import torch
import torchvision
from PIL import Image

import os
import matplotlib.pyplot as plt

# The size is 64 x 64 now

from skimage import io
path = 'batch1/part_1/'
for item in os.listdir(path):
        if os.path.isfile(path+item):
            if(path+item == 'batch1/part_1/.DS_Store'):
                continue
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            imResize = im.resize((64,64), Image.Resampling.LANCZOS)
            imResize.save(path + 'new/' + item, 'JPEG')
            
            


In [14]:

path = 'batch1/part_1/new/'
img_names = os.listdir(path)

images = np.empty(shape = (10025, 64, 64, 3))
for idx, name in enumerate(img_names):
    img_name = path + name
    # Use you favourite library to load the image
    image = plt.imread(img_name)
    images[idx] = image
    

In [79]:
# print(annotation_batch1.head())
import pandas as pd

position_maps= {"standing": 0, 
                "takedown1": 1,
                "takedown2": 2,
                "open_guard1": 3,
                "open_guard2": 4,
                "half_guard1": 5,
                "half_guard2": 6,
                "closed_guard1": 7,
                "closed_guard2": 8,
                "5050_guard": 9,
                "mount1": 10,
                "mount2": 11,
                "back1": 12,
                "back2": 13,
                "turtle1": 14,
                "turtle2": 15,               
                "side_control1" : 16,
                "side_control2" : 17}


annotation['position_num'] = [0]*len(annotation['position'])
for i in range(10025):
#     print(annotation["position"][i])
    annotation['position_num'][i] = position_maps[annotation["position"][i]]
annotation_batch1 = annotation['position_num'][:10025]


annotation_batch1 = torch.from_numpy(annotation_batch1.values)


from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(
    images, annotation_batch1, test_size=0.25, random_state=42)

train_X = torch.from_numpy(train_X)
train_X = torch.movedim(train_X, source=3, destination=1)

test_X = torch.from_numpy(test_X)
test_X = torch.movedim(test_X, source=3, destination=1)



/var/folders/rv/qhv_6w4n707gq2vr6xwbmj9w0000gn/T/ipykernel_65150/4127807505.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotation['position_num'][i] = position_maps[annotation["position"][i]]


In [71]:
## Compose Transformations
from torchvision import transforms
data_transforms = transforms.Compose([
        transforms.GaussianBlur(kernel_size=(3,3), sigma=(0.1, 5)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=(0, 180))
])

from torch import nn
class my_net(nn.Module):
    
    ## Constructor commands
    def __init__(self):
        super(my_net, self).__init__()
        
        ## Define architecture
        self.conv_stack = nn.Sequential(
            nn.Conv2d(3,8,3,1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(8,16,2,1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Flatten(),
            nn.Linear(3600, 200),
            nn.ReLU(),
            nn.Linear(200, 18)
        )
    
    ## Function to generate predictions
    def forward(self, x):
        scores = self.conv_stack(x)
        return scores
    
    
    

In [81]:
# Part D
## Hyperparms
epochs = 300
lrate = 0.001
bsize = 28

## For reproduction purposes 
torch.manual_seed(3)

## Cost Function
cost_fn = nn.CrossEntropyLoss()

## Intialize the model
net = my_net()

## Optimizer (Stochastic Gradient Descent)
optimizer = torch.optim.SGD(net.parameters(), lr=lrate)



## Make DataLoader
from torch.utils.data import DataLoader, TensorDataset
y_tensor = torch.Tensor(train_y)
train_loader = DataLoader(TensorDataset(train_X.type(torch.FloatTensor), 
                        y_tensor.type(torch.LongTensor)), batch_size=bsize)

## Re-run the training loop, notice the new data_transforms() command
track_cost = np.zeros(epochs)
cur_cost = 0.0

for epoch in range(epochs):
    cur_cost = 0.0
    correct = 0.0
    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        
        ## Transform the input data using our data augmentation strategies
        inputs = data_transforms(inputs)
        
        ## Same as before
        optimizer.zero_grad()
        outputs = net(inputs)
        cost = cost_fn(nn.Softmax(dim=1)(outputs), labels)
        cost.backward()
        optimizer.step()
        cur_cost += cost.item()
    
    ## Store the accumulated cost at each epoch
    track_cost[epoch] = cur_cost
#     print(f"Epoch: {epoch} Cost: {cur_cost}") ## Uncomment this if you want printed updates
    

## Initialize objects for counting correct/total 
correct = 0
total = 0

# Specify no changes to the gradient in the subsequent steps (since we're not using these data for training)
with torch.no_grad():
    for data in train_loader:
        # Current batch of data
        images, labels = data
        
        # pass each batch into the network
        outputs = net(images)
        
        # the class with the maximum score is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        
        # add size of the current batch
        total += labels.size(0)
        
        # add the number of correct predictions in the current batch
        correct += (predicted == labels).sum().item()

## Calculate and print the proportion correct
print(f"Training Accuracy is {correct/total}")


KeyboardInterrupt: 